# 01_compute_headmotion

Compute head movement and output the data for visualization

In [1]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack

In [2]:
# define variables
sub_list = ['{0:0>2d}'.format(sub) for sub in np.arange(1,12)]
run_list = ['{0:0>2d}'.format(run) for run in np.arange(1,9)]

# set path
bids_root = '/nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids'
results_pth = '/nfs/e5/studyforrest/forrest_movie_meg/tech_val_results'
if os.path.exists(results_pth) is False:
    os.mkdir(results_pth)

# set fiducials channel
coord = {'nas' : ['HLC0011','HLC0012','HLC0013'], 
         'lpa' : ['HLC0021','HLC0022','HLC0023'], 
         'rpa' : ['HLC0031','HLC0032','HLC0033']}

In [3]:
def extract_hpi(raw_data):
    '''
    Extract hpi data from mne raw object.
    Returns
    -------
    hpi_data : Dict
        {channel_name: hpi_value}
    '''
    picks = mne.pick_channels_regexp(raw_data.ch_names, regexp='HLC00[123][123]...')
    hpi_data = raw_data.get_data(picks=picks)
    
    hpi = {}
    for i, pick in enumerate(picks):
        hpi[raw_data.ch_names[pick].split('-')[0]] = hpi_data[i]
            
    return hpi

In [4]:
def compute_headmotion(meg_data, coord):
    '''
    compute headmotion distance from mne raw object.
    Parameters
    ----------
    meg_data: mne object
    coord: Dict
        {fiducial: [channel name of x, channel name of y, channel name of z]}
    
    Returns
    -------
    head_movement : array
        [n_fiducial, n_sample]
    '''
    
    raw_data = raw.copy().crop(tmin=raw.annotations.onset[0], tmax=raw.annotations.onset[-1])
    hpi = extract_hpi(raw_data)
    
    # get relative positions to initial positions
    hpi_de = {}
    for chn, val in hpi.items():
        hpi_de[chn] = 1000*(val-val[0])
    
    # compute head motion
    head_movement = [np.sqrt(hpi_de[coord[fiducial][0]]**2 + 
                             hpi_de[coord[fiducial][1]]**2 + 
                             hpi_de[coord[fiducial][2]]**2) 
                     for fiducial in coord.keys()]
    
    return np.asarray(head_movement)

In [6]:
# get headmotion distances
head_motion = {}
for sub in sub_list:
    
    # get runlist
    head_motion[sub] = []
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
        
    for run in run_ls:
        # get hpi data
        sub_path = BIDSPath(subject=sub, run=int(run), task='movie', session='movie', root=bids_root)
        raw = mne_bids.read_raw_bids(sub_path)
        
        head_motion[sub].append(compute_headmotion(raw, coord))


ds directory : /nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       4.65   74.88    0.00 mm <->    4.65   74.88    0.00 mm (orig :  -65.68   46.24 -249.17 mm) diff =    0.000 mm
      -4.65  -74.88    0.00 mm <->   -4.65  -74.88    0.00 mm (orig :   42.00  -58.24 -250.44 mm) diff =    0.000 mm
      92.94    0.00    0.00 mm <->   92.94   -0.00    0.00 mm (orig :   46.46   62.07 -225.18 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-01_meg.ds/sub-01_ses-movie_task-movie_run-01_meg.meg4: 
    System clock channel is availabl

KeyboardInterrupt: 

In [7]:
# convert head motion data to 1000 bins
# get max head movement
upper_bound = np.ceil(max([run.max() for sub, sub_data in head_motion.items() for run in sub_data]))

# binning
hm = {}
for sub in sub_list:
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
    
    hm_sub = []
    for run in run_ls:
        
        mv = head_motion[sub][int(run)-1]
        hm_bins = [pd.cut(mv[i], np.linspace(0, upper_bound, 1000)).value_counts().values 
                   for i in np.arange(mv.shape[0])]
        
        hm_sub.append(np.asarray(hm_bins))
        
    hm[sub] = hm_sub
    print( sub + ' done')
        

IndexError: list index out of range

In [77]:
# save hpi data
for sub in list(hm.keys())[1:]:
    hm[sub].append(np.nan)
df = pd.DataFrame(hm, columns=sub_list, index=run_list+['09'])
df.to_pickle(pjoin(results_pth, 'head_motion.pickle'))

np.save(pjoin(results_pth, 'head_motion_bins'), np.linspace(0, upper_bound, 1000))